In [1]:
import os
from sys import argv
import time
from novaclient.client import Client

config = {'username':os.environ['OS_USERNAME'],
          'api_key':os.environ['OS_PASSWORD'],
          'project_id':os.environ['OS_TENANT_NAME'],
          'auth_url':os.environ['OS_AUTH_URL']}

nc = Client('2',**config)
ubuntu = 'Ubuntu Server 14.04 LTS (Trusty Tahr)'
image = nc.images.find(name=ubuntu)
flavor = nc.flavors.find(name='m1.medium')
network = nc.networks.find(label='ACC-Course-net')
userdata = open('userdata-project.yml', 'r')
name = argv[1]

try:
    server = nc.servers.create(name=name, image=image.id, flavor=flavor.id,
                               network=network.id, key_name='haadkey',
                               userdata=userdata, security_groups=None)
finally:
    userdata.close()

floating_ips = nc.floating_ips.list()
state = server.status

while(state == 'BUILD'):
    time.sleep(4)
    server = nc.servers.get(server.id)
    state = server.status

for ip in floating_ips:
    if ip.instance_id == None:
        server.add_floating_ip(ip)
        print 'Ubuntu Server running at: {}'.format(ip.ip)
        break
else:
    print 'No available floating IPs in the pool'


Ubuntu Server running at: 130.238.29.107


In [7]:
import uuid
import swiftclient.client
config = {'user':os.environ['OS_USERNAME'], 
          'key':os.environ['OS_PASSWORD'],
          'tenant_name':os.environ['OS_TENANT_NAME'],
          'authurl':os.environ['OS_AUTH_URL']}


conn = swiftclient.client.Connection(auth_version=2, **config)
tmp = conn.get_container("tweets")
for num in range(0,len(tmp[1])):
    print(tmp[1][num]['name'])
    #print(tmp[1][num])
    

tweets_0.txt
tweets_1.txt
tweets_10.txt
tweets_11.txt
tweets_12.txt
tweets_13.txt
tweets_14.txt
tweets_15.txt
tweets_16.txt
tweets_17.txt
tweets_18.txt
tweets_19.txt
tweets_2.txt
tweets_3.txt
tweets_4.txt
tweets_5.txt
tweets_6.txt
tweets_7.txt
tweets_8.txt
tweets_9.txt


In [8]:
a = conn.get_object("tweets", 'tweets_0.txt')


In [9]:
from celery import Celery
import json
from collections import Counter

app = Celery('tasks', backend='rpc://', broker='amqp://')

@app.task
def getJson():
    #with open("/home/hampus/skola/CloudComputing/FilesFromAssignments2015-09-15/C3/localSolution/tweets.txt") as f:
    content = a #f.readlines()
    tmp = runme.delay(content)
    return tmp


# INPUT = list 
# Varje rad motsvarar en array. varannat.
@app.task
def runme(Jdata):
    tmp = Counter()

    for item in Jdata:
        if(1<len(item)): # Removes Empty rows
            tmp = tmp + manageRow.delay(item)            
    return(tmp)

@app.task
def manageRow(item):
    result = Counter()
    data = json.loads(item)
    if('retweeted_status' not in data.keys()): #Removes retweets
        result  = Counter(data['text'].split())
    return result

#runme(getJson())


In [11]:
tmp = getJson.delay()

In [13]:
tmp.ready()

False

In [ ]:
print(tmp.get())